In [1]:
from utils.download import download_all
from utils.download import get_stock_by_range
from utils.features import add_features
from datetime import date
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
def PlotSignal(input_signal_df: pd.DataFrame, signal_field: str):
    fig = go.Figure()

    # Create subplots
    fig = make_subplots(
        rows=2, 
        cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.05,  # Adjust space between the plots
        row_heights=[0.6, 0.4]  # Adjust heights of the subplots
    )


    buy_signals = input_signal_df[input_signal_df[signal_field] == "buy"]
    sell_signals = input_signal_df[input_signal_df[signal_field] == "sell"]

    cover_buy_signals = input_signal_df[input_signal_df[signal_field] == "cover/buy"]
    sell_short_signals = input_signal_df[input_signal_df[signal_field] == "sell/short"]

    # Add price data
    fig.add_trace(go.Scatter(x=input_signal_df['stock_date'], 
                             y=input_signal_df['close'], 
                             name='Close Price'),
                            row=1,
                            col=1)    
    
    # fig.add_trace(go.Scatter(x=input_signal_df['stock_date'], 
    #                          y=input_signal_df['mean_close'], 
    #                          name='Mean Close Price'),
    #                         row=1,
    #                         col=1)    
    
    # fig.add_trace(go.Scatter(x=input_signal_df['stock_date'], 
    #                          y=input_signal_df['macd_signal_line'], 
    #                          name='macd_signal_line'),
    #                         row=1,
    #                         col=1)
    
    # fig.add_trace(go.Scatter(x=input_signal_df['stock_date'], 
    #                          y=input_signal_df['macd'], 
    #                          name='macd'),
    #                         row=1,
    #                         col=1)        
    
    # Add buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals['stock_date'],
        y=buy_signals['close'],
        mode='markers',
        name='Buy Signal',
        marker_symbol='triangle-up',
        marker_color='green',
        marker_size=10
    ),
        row=1,
        col=1)

    # Add sell signals
    fig.add_trace(go.Scatter(
            x=sell_signals['stock_date'],
            y=sell_signals['close'],
            mode='markers',
            name='Sell Signal',
            marker_symbol='triangle-down',
            marker_color='red',
            marker_size=10
        ),  
            row=1, 
            col=1
        )
    

     # Add buy signals
    fig.add_trace(go.Scatter(
        x=cover_buy_signals['stock_date'],
        y=cover_buy_signals['close'],
        mode='markers',
        name='Cover buy',
        marker_symbol='arrow-up-open',
        marker_color='green',
        marker_size=10
    ),
        row=1,
        col=1)

    # Add sell signals
    fig.add_trace(go.Scatter(
            x=sell_short_signals['stock_date'],
            y=sell_short_signals['close'],
            mode='markers',
            name='Sell Short',
            marker_symbol='arrow-down-open',
            marker_color='red',
            marker_size=10
        ),  
            row=1, 
            col=1
        )
    
    fig.add_trace(
    go.Bar(x=input_signal_df['stock_date'], 
           y=input_signal_df['volume'],
           name='Volume'),
        row=2, col=1
    )
    

    # Update layout
    fig.update_layout(
        title='Buy and Sell Signals',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        height=1000
    )

    fig.show()

In [3]:
download_all()

index alredy exists
2025-02-03 00:00:00 -> AAPL


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


2025-02-03 00:00:00 -> GOOG


[*********************100%***********************]  1 of 1 completed

2025-02-03 00:00:00 -> AMZN


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


2025-02-03 00:00:00 -> AMD
2025-02-03 00:00:00 -> NVDA


[*********************100%***********************]  1 of 1 completed


2025-02-03 00:00:00 -> IBM


[*********************100%***********************]  1 of 1 completed


In [3]:
df = get_stock_by_range(date(2020, 1, 1), date(2025, 2,28))

index alredy exists
select * from stock_data where stock_date >= '2020-01-01' and stock_date <= '2025-02-28'


In [11]:
df = add_features(df)

In [12]:
signals_df = df[df['ticker'] == 'AAPL']
PlotSignal(signals_df, 'signal_command')

In [13]:
df2 = signals_df[signals_df['signal_command'] != 'hold']

In [14]:
for i in range(1, len(df2)):
    # Используем iloc, чтобы брать i-ю строку вне зависимости от значения индекса
    signal_now = df2.iloc[i]['signal_command']
    signal_prev = df2.iloc[i-1]['signal_command']
    if signal_now != 'buy' and signal_now != 'sell' and signal_prev != 'buy' and signal_prev != 'sell' and signal_now != signal_prev:
        price_change = abs(df2.iloc[i]['mean_close'] - df2.iloc[i-1]['mean_close'])/df2.iloc[i]['mean_close']
        if price_change < 0.1:
            if signal_now == 'sell/short':
                   df2.iloc[i, df.columns.get_loc('signal_command')] = 'sell'
            if signal_now == 'cover/buy':
                   df2.iloc[i, df.columns.get_loc('signal_command')] = 'buy'

In [15]:
signals_df.update(df2)

In [16]:
PlotSignal(signals_df, 'signal_command')